In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
df_train = pd.read_csv('../input/train.csv')

In [ ]:
df_test = pd.read_csv('../input/test.csv')

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
## as we have many column, set below properties, else all column will not be shown when you will describe
pd.set_option('display.max_columns', None) 

In [ ]:
df_train.describe()

#### No attribute is missing as count is 15120 for all attributes. Hence, all rows can be used
#### Soil_Type7 and Soil_Type15 are constant. we can drop
#### Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways --skewed
#### Vertical_Distance_To_Hydrology is negetive.
#### Wilderness_Area and Soil_Type are one hot encoded. Hence, they could be converted back for some analysis¶

In [ ]:
df_train.info()

In [ ]:
## check if any column is null
df_train.isnull().sum().any()

In [ ]:
df_train.columns

In [ ]:
df_train.Cover_Type.value_counts()
## we can see 7 cover types are there and count is also same

#### Get the top 5 correlated feature

In [ ]:
data_corr = df_train.corr()
size=10
threshold=0.5
#create a dataframe with only 'size' features
data=df_train.iloc[:,:size] 

#get the names of all the columns
cols=data.columns 
corr_list=[]
for i in range(0,size):
    for j in range(i+1, size):
        if (data_corr.iloc[i,j] >= threshold and data_corr.iloc[i,j] < 1) or (data_corr.iloc[i,j] < 0 and data_corr.iloc[i,j] <= -threshold):
            corr_list.append([data_corr.iloc[i,j],i,j])

In [ ]:
#Sort to show higher ones first            
s_corr_list = sorted(corr_list,key=lambda x: -abs(x[0]))

#Print correlations and column names
for v,i,j in s_corr_list:
    print ("%s and %s = %.2f" % (cols[i],cols[j],v))

In [ ]:
## check the skewness of the data
df_train.skew()

### Visualization

In [ ]:
col_list=df_train.columns
col_list = [col for col in col_list if not 'Soil' in col]

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_train[col_list].corr(),annot=True)

In [ ]:
sns.pairplot(df_train[col_list],hue='Cover_Type')

#### Comapre the model which gives best accuracy with this data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.model_selection import cross_val_score

In [ ]:
X = df_train.drop(['Id','Cover_Type'],axis=1)
y = df_train['Cover_Type']

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
models = []
models.append(('LR',LogisticRegression()))
models.append(('DT',DecisionTreeClassifier()))
models.append(('GB',GaussianNB()))
models.append(('RFC',RandomForestClassifier()))

results=[]
names=[]
scoring='accuracy'
for name,model in models:
    kfold = model_selection.KFold(n_splits=20,random_state=12345)
    cv_results = model_selection.cross_val_score(model,X, y, cv=kfold,scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name,cv_results.mean(),cv_results.std())
    print(msg)

### Random Forest model gives best accuracy than any other model

In [ ]:
rf = RandomForestClassifier()

In [ ]:
train_X = df_train.drop(['Id','Cover_Type'],axis=1)
train_Y = df_train['Cover_Type']
test_x = df_test.drop('Id',axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
##Split the training set into training and validation sets
X_train,X_test,Y_train,Y_test = train_test_split(train_X,train_Y,test_size=0.2)

In [ ]:
rf.fit(X_train,Y_train)

In [ ]:
y_predict = rf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(Y_test,y_predict))
print(accuracy_score(Y_test,y_predict))

In [ ]:

test_id = pd.DataFrame(df_test.Id)

In [ ]:
test_id.head()

In [ ]:
test_id['Cover_Type']=rf.predict(df_test.drop('Id',axis=1))

In [ ]:
test_id.head()

In [ ]:
data_to_submit = pd.DataFrame({
    'Id':test_id['Id'],
    'Cover_Type':test_id['Cover_Type']
})

In [ ]:
data_to_submit.head()

In [ ]:
data_to_submit.to_csv('Forest_Cover_Type.submit.csv', index = False)